In [1]:
#import the relevant libraries
import json
import spacy as sp
from spacy import displacy as d

In [2]:
#import singe movie from the data set for analysis
data = {}
with open('movie.json') as json_file:  
    data = json.load(json_file)

In [3]:
#use spacy to check the relevant entities that are tagged
nlp = sp.load('en_core_web_sm')
movieText = nlp(data['Summary'])
d.render(movieText,style='ent', jupyter = True)

In [4]:
#print the entities
for word in movieText.ents:
    print(word.text , word.label_)

Ger PERSON
Die 1000 Augen WORK_OF_ART
Mabuse PERSON
1960 DATE
West Germany GPE
West German NORP
French NORP
Fritz Lang PERSON
Peter van Eyck PERSON
Dawn Addams PERSON
Gert Fröbe PERSON
Mabuse PERSON
Lang PERSON
1922 DATE
1933 DATE
The Thousand Eyes of Dr. Mabuse ORG
German NORP
Mabuse WORK_OF_ART
the following years DATE
Rialto Film PERSON
Krimi PERSON
Inspector Kras ORG
Peter Cornelius PERSON
Henry Travers PERSON
American NORP
the Luxor Hotel ORG
Nazis NORP
World War II EVENT
Marian Menil PERSON
Hieronymus B. Mistelzweig PERSON
Mabuse PERSON
The Thousand Eyes of Dr. Mabuse ORG
CCC Filmkunst ORG
West Germany GPE
Incom PERSON
Italy GPE
Critérion Film ORG
France GPE
Die 1000 Augen WORK_OF_ART
Mabuse PERSON
German NORP
Mabuse PERSON
Italian NORP
Mabuse ORG
French).,[4 CARDINAL
Fritz Lang PERSON
U.S. GPE
Germany GPE
three CARDINAL
Artur Brauner ORG
The Tiger of Eschnapur ORG
The Indian Tomb LOC
The Thousand Eyes of Dr. Mabuse ORG
Mabuse PERSON
Norbert Jacques PERSON
Lang PERSON
two CARDINA

In [4]:
#render the tree strcuture to analyse the realtionship between entities and different part of speech.
d.render(movieText, style="dep",jupyter = True)

In [5]:
#used to extract the realtion from the tagged text
def extract_relations(doc):
    relations = []
    spans = list(doc.ents) + list(doc.noun_chunks)
    for span in spans:
        span.merge()

    def extract_person_realtions():
        for word in filter(lambda w: w.ent_type_ == "PERSON" and w.dep_ != "nsubj", doc):
            subject = []
            if word.dep_ == "pobj":
                subject = [w for w in word.head.head.subtree if w.pos_ == 'VERB']
            elif word.dep_ == "dobj" or word.dep_ == "appos" or word.dep_ == 'npadvmod' :
                subject = [w for w in word.head.subtree if w.pos_ == 'VERB']
        
            if subject:
                subject = subject[0]
                relations.append((subject, word))
        
    def extract_budget_or_date_relation(tag):
        for word in filter(lambda w: w.ent_type_ == tag, doc):
            if word.dep_ in ("attr", "dobj"):
                subject = [w for w in word.head.lefts if w.dep_ == "nsubj"]
                if subject:
                    subject = subject[0]
                    relations.append((subject, word))
            elif word.dep_ == "pobj" and word.head.dep_ == "prep":
                relations.append((word.head.head, word))

    extract_budget_or_date_relation("DATE")
    extract_budget_or_date_relation("MONEY")
    extract_person_realtions()
    
    return relations        


In [6]:
#extract the relation and print the same for analysis of a single movie
relations = extract_relations(movieText)
for r1, r2 in relations:
    print("{:<10}\t{}\t{}".format(r1.text, r2.ent_type_, r2.text))

a boy     	DATE	1969
find      	DATE	1995
released  	DATE	December 15, 1995
the ,10th highest-grossing film	DATE	1995
released  	DATE	2005
bury      	DATE	1869
return    	DATE	1969
a budget  	MONEY	approximately $65 million
directed  	PERSON	Joe Johnston
was       	PERSON	Van Allsburg
was       	PERSON	Greg Taylor
tracking  	PERSON	Sarah
marketed  	PERSON	Zathura
marketed  	PERSON	Jumanji
meets     	PERSON	Carl Bentley
attending 	PERSON	Sam
makes     	PERSON	Carl
tells     	PERSON	Alan
search    	PERSON	Alan
need      	PERSON	Sarah
is        	PERSON	Carl
set       	PERSON	Van Pelt
set       	PERSON	Alan
accepts   	PERSON	Alan
meet      	PERSON	Judy
hear      	PERSON	Jumanji
lives     	PERSON	Chris Van Allsburg


In [5]:
#movie node to hold the data extracted from all the movies
class Movie(object):
    def __init__(self, name , relations):
        #unique identifier of the node
        
        self.name = str(name)
        self.relations = relations

In [8]:
# class Node(object):
#     def __init__(self, name , nodeType):
#         #unique identifier of the node
#         self.name = str(name)
#         #type of node namely movie , person , date , money 
#         self.nodeType = nodeType
#         #list of tuple (relation , node)
#         self.neighbors = []
        
        
# #dictionary of nodes - class Node
# node_dict = {}

# #list of all movies including the relation
# movies = []


# #add a new movie to movie list
# movie = Movie(data['title'],relations)
# movies.append(movie)

# #loop over all movies and create nodes of graph
# for movie in movies:
#     node1 = ''
#     #mostlty only movie node will have neighbors as the infomation is fetched for movies.
#     if movie.name in node_dict:
#         node1 = node_dict[movie.name]
#     else:
#         node1 = Node(movie.name , "Movie")
#         node_dict[node1.name] = node1
#     for r1, r2 in movie.relations:
#         #node1 will be movie
#         #r1.text --> relation
#         #r2.ent_type_ --> type of node2
#         #r2.text --> name of node2
#         node2 = ''
#         if r2.text in node_dict:
#             node2 = node_dict[r2.text]
#         else:
#             node2 = Node(r2.text , r2.ent_type_)
#             node_dict[node2.name] = node2
#         node1.neighbors.append((r1.text,node2))

# from py2neo import Graph
# from py2neo import Node, Relationship
# graph = Graph(password="adbi")
# graph.delete_all()


# for key,node1 in node_dict.items():
#     n1 = Node(node1.nodeType, name=node1.name)
#     for neighbor in node1.neighbors:
#         relation = neighbor[0]
#         node2 = neighbor[1]
#         n2 = Node(node2.nodeType, name=node2.name)
#         relationship = Relationship(n1,relation,n2)
#         graph.create(relationship)


# ###### test code for buildng the logic for relation extraction
# relations = []
# for money in filter(lambda w: w.ent_type_ == "PERSON" and w.dep_ != "nsubj", movieText):
#     subject = []
#     print(money)
#     print(money.dep_)
#     if money.dep_ == "pobj":
#         subject = [w for w in money.head.head.subtree if w.pos_ == 'VERB']
#     elif money.dep_ == "dobj" or money.dep_ == "appos" or money.dep_ == 'npadvmod' :
#         subject = [w for w in money.head.subtree if w.pos_ == 'VERB']
#     if subject:
#         print(subject)
#         subject = subject[0]
#         relations.append((subject, money))
# print(relations)

In [8]:
#extract the relationshi for all the movies and store the movie in neo4j-grpah database.
from py2neo import Graph
from py2neo import Node, Relationship

nlp = sp.load('en_core_web_sm')

def read_full_data_and_create_movie_list():
    movies = {}
    movies_node = []
    with open('movies.json') as json_file:  
        movies = json.load(json_file)
    
    for i in range(0,len(movies)):
        data = movies[i]
        print(data['title'])
        movieText = nlp(data['Summary'])
        relations = extract_relations(movieText)
        movie = Movie(data['title'],relations)
        movies_node.append(movie) 
    return movies_node

def create_graph_in_neo4j_from_movies(movies):
    #dictionary of nodes - class Node neo4j
    node_dict = {}

    graph = Graph(password="adbi")
    graph.delete_all()
    #loop over all movies and create nodes of graph
    for movie in movies:
        node1 = ''
        #mostlty only movie node will have neighbors as the infomation is fetched for movies.
        if movie.name in node_dict:
            node1 = node_dict[movie.name]
        else:
            node1 = Node("Movie", name=movie.name)
            node_dict[movie.name] = node1
        for r1, r2 in movie.relations:
            #node1 will be movie
            #r1.text --> relation
            #r2.ent_type_ --> type of node2
            #r2.text --> name of node2
            node2 = ''
            if r2.text in node_dict:
                node2 = node_dict[r2.text]
            else:
                node2 = Node(r2.ent_type_, name=r2.text)
                node_dict[r2.text] = node2
            relationship = Relationship(node1,r1.text,node2)
            graph.create(relationship)
    return graph
    
#graph = create_graph_in_neo4j_from_movies(movies)

            

In [9]:
#extract relations from the complete dataset
node = read_full_data_and_create_movie_list()

Jumanji
Tom and Huck
GoldenEye
Father of the Bride Part II
Grumpier Old Men
Waiting to Exhale
Toy Story
Cutthroat Island
The American President
Dracula: Dead and Loving It
Four Rooms
Money Train
Ace Ventura: When Nature Calls
Leaving Las Vegas
The City of Lost Children
12 Monkeys
Dangerous Minds
Across the Sea of Time
Dead Presidents
To Die For
When Night Is Falling
The Usual Suspects
Mighty Aphrodite
Lamerica
The Big Green
Il Postino: The Postman
Kids of the Round Table
Mr. Holland's Opus
Bio-Dome
Don't Be a Menace to South Central While Drinking Your Juice in the Hood
Two If by Sea
Lawnmower Man 2: Beyond Cyberspace
Two Bits
French Twist  (film)
From Dusk till Dawn
The Crossing Guard
The Juror
Things to Do in Denver When You're Dead
Once Upon a Time...When We Were Colored
Dunston Checks In
Last Summer in the Hamptons
Angels & Insects
La haine
The Journey of August King
In the Bleak Midwinter  (film)
Vampire in Brooklyn
Heidi Fleiss: Hollywood Madam
Mr. Wrong
Muppet Treasure Island
Br

In [10]:
len(node)

560

In [11]:
#create graph for all mvoies
graph = create_graph_in_neo4j_from_movies(node)